In [1]:
# automatically upload modules
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
from argparse import Namespace
import os, sys
import pandas as pd
#import pickle
from ray import tune
import ConfigSpace as CS
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.schedulers import HyperBandForBOHB
import torch
import argparse # for ray distributed training


from genome_embeddings import data_viz
from genome_embeddings import evaluate
from genome_embeddings import models
from genome_embeddings import train_test
from genome_embeddings import util
from genome_embeddings import trainable # import before ray (?)
import ray

In [ ]:
os.system("rm file_out")
os.system("rm file_err")

sys.stdout = open('file_out', 'w')
sys.stderr = open('file_err', 'w')

In [4]:
settings = Namespace(
    DATA_FP = '/home/ndudek/projects/def-dprecup/ndudek/',
    SAVE_FP = '/home/ndudek/projects/def-dprecup/ndudek/',
    num_epochs = 2,
    num_cpus=1)

# settings = Namespace(
#     DATA_FP = '/Users/natasha/Desktop/mcgill_postdoc/ncbi_genomes/genome_embeddings/data/', 
#     SAVE_FP = '/Users/natasha/Desktop/mcgill_postdoc/ncbi_genomes/genome_embeddings/',
#     num_epochs = 2,
#     num_cpus=5)

In [5]:
flags = Namespace(
    KEGG_FP = '/Users/natasha/Desktop/mcgill_postdoc/ncbi_genomes/kegg_dataset/',
    data_source = 'kegg', #['get_homologues' | 'kegg']
    n_test = 0.1, # train-test split, n * 100 = % of data that goes into test set (e.g.: 0.1 -> 10%)
    batch_size = 128,
    lr = 1e-3,
    kfolds = 10, # number of folds for cross-validation
    print_every = 50, # print loss every n batches during training (5)
    replacement_threshold = 0.5, # probability over which binarizer converts to a 1
    num_corruptions = 100, # number of corrupted versions of a genome to produce
    corruption_fraction = 0.5, # fraction of genes to retain during corruption process
    phy_mode = "bacteria_only", # training with only bacteria vs also euk/arch
    cirriculum = False, # implement cirriculum learning based on gene count
    rare_threshold = 10, # drop features that occur fewer than this times in training ds 
    weight_decay=0.1 # L2 regularization
    )

In [ ]:
print('done loading modules and setting Namespace variables')

### Data exploration + preprocessing 

In [ ]:
# # First create genome representations (very slow)
# # Each genome is a list of KO's and/or KEGG modules
# if os.path.isfile(settings.DATA_FP+'genome_to_mod.csv'):
#     print("Genome representations already exist")
# else:
#     genome_rep.genome_kos(flags.KEGG_FP)
#     print("Must generate genome representations from scratch. This will take several hours.")

In [ ]:
df, cluster_names = util.load_data(settings.DATA_FP, flags.data_source)
genome_to_tax = util.genome_to_tax(df)
np.save(settings.DATA_FP+'genome_to_tax.npy', genome_to_tax) 

In [ ]:
#data_viz.tax_distrib(df, genome_to_tax)

In [ ]:
#data_viz.module_stats(df)

In [ ]:
#data_viz.genes_per_genome(df)

In [ ]:
# genome_to_tax = util.genome_to_tax(df)

In [ ]:
# train_orig = pd.read_csv(settings.DATA_FP+"uncorrupted_train_balanced.csv", index_col=0)

In [ ]:
# train_genomes = list(train_orig.index)

In [ ]:
# Split train-test sets in a phylogenetically balanced manner 
if os.path.isfile(settings.DATA_FP+'uncorrupted_train_balanced.csv'):
    print("Train-test split already exists, loading from file")
    train_orig = pd.read_csv(settings.DATA_FP+"uncorrupted_train_balanced.csv", index_col=0)    
    test_orig = pd.read_csv(settings.DATA_FP+"uncorrupted_test_balanced.csv", index_col=0)    

else:
    # Create dict mapping each genome to a unique numerical ID
    genome_to_num ={}
    for i,genome in enumerate(df.index):
        genome_to_num[genome] = i

    num_to_genome = {v: k for k, v in genome_to_num.items()}
        
    print("Generating train-test split")
    train_orig, test_orig = util.balanced_split(df, flags.n_test, genome_to_tax, 
                                                num_to_genome, settings.DATA_FP)    
    train_orig.to_csv(settings.DATA_FP+'uncorrupted_train_balanced.csv')
    test_orig.to_csv(settings.DATA_FP+'uncorrupted_test_balanced.csv')

In [ ]:
#data_viz.hist_prob_ko(train_orig)

In [ ]:
if flags.phy_mode == "bacteria_only":
    train_genomes = train_orig.index.to_list()
    test_genomes = test_orig.index.to_list()
    
    unf_train_data, train_tax_dict = util.bacteria_only(train_orig, train_genomes, genome_to_tax)
    unf_test_data, test_tax_dict = util.bacteria_only(test_orig, test_genomes, genome_to_tax)

In [ ]:
# Remove rare features from train + test datasets
# Rare = fewer than n occurences in training dataset
# Last argument specifies n, set to correspond to 1% of genomes (3432 genomes -> n = 34)
# Remove genes occuring in <1.1% of genomes ---> extra 0.1 is to make there be an even number of features
#     An even number of features is essential for having the autoencoder layers work out properly
train_data, test_data, cluster_names = util.remove_rare(unf_train_data, unf_test_data, 
                                                        cluster_names, unf_train_data.shape[0]*0.01)

In [6]:
# Produce corrupted genomes
# Could eventually do re-sampling / extra-corrupting to have more examples of "rare" genome types
#    e.g.: those from underrepresented groups M00003   

if os.path.isfile(settings.DATA_FP+'corrupted_train_07-17-20.pt'):
    print("Corrupted genomes already exist")
    train_data = torch.load(settings.DATA_FP+"corrupted_train_07-17-20.pt")
    test_data = torch.load(settings.DATA_FP+"corrupted_test_07-17-20.pt")
    genome_idx_train = torch.load(settings.DATA_FP+"genome_idx_train_07-17-20.pt")
    genome_idx_test = torch.load(settings.DATA_FP+"genome_idx_test_07-17-20.pt")
else:
    print("Generating corrupted dataset from scratch with",flags.num_corruptions,"corrupted versions of each genome")
    train_data, genome_idx_train = util.corrupt(train_data, flags.num_corruptions, flags.corruption_fraction, 
                                                    cluster_names, "train", settings.DATA_FP)
    print("Finished training data, starting test")
    test_data, genome_idx_test = util.corrupt(test_data, flags.num_corruptions, flags.corruption_fraction, 
                                                  cluster_names, "test", settings.DATA_FP)

Corrupted genomes already exist


In [ ]:
sys.stdout.flush()
sys.stderr.flush()

In [ ]:
#import numpy as np # this is slooooow
#np.savetxt(settings.DATA_FP+"corrupted_train_1407.txt", train_data.numpy())
#np.savetxt(settings.DATA_FP+"corrupted_test_1407.txt", test_data.numpy())

In [ ]:
# print(("There are %s genomes and %s features in the training dataset") % 
#       (train_data.shape[0],int(train_data.shape[1]/2)))

# print(("There are %s genomes and %s features in the test dataset") % 
#       (test_data.shape[0],int(test_data.shape[1]/2)))

### Define and train network

In [ ]:
# print(train_data.shape[1])

In [7]:
# num_features = int(train_data.shape[1]/2) # Number of features in the entire dataset (train + test)

In [28]:
# # define the network
# model = models.AutoEncoder(num_features, 2)
# print(model)

AutoEncoder(
  (layers): ModuleList(
    (0): Linear(in_features=7065, out_features=13, bias=True)
    (1): Linear(in_features=13, out_features=4, bias=True)
    (2): Linear(in_features=4, out_features=13, bias=True)
    (3): Linear(in_features=13, out_features=7065, bias=True)
  )
)


In [ ]:
sys.stdout.flush()
sys.stderr.flush()

In [ ]:
config_space = CS.ConfigurationSpace()

config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(name='nn_layers', choices=[1, 2])) #, 3, 4])) 

config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(name='batch_size', choices=[32])) #, 64, 128, 256]))

#Optimizer = Adam -- LR less important
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter('lr', lower=1e-4, upper=1e-1, log=True))

config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter('weight_decay', lower=1e-5, upper=1e-2, log=True))

algo = TuneBOHB(
    config_space, max_concurrent=4, metric='test_f1', mode='max')

bohb = HyperBandForBOHB(
    time_attr='training_iteration',
    metric='test_f1',
    mode='max',
    max_t=100, 
    reduction_factor=3)

In [ ]:
memory = 2000 * 1024 * 1024
object_store_memory = 200 * 1024 * 1024
driver_object_store_memory=100 * 1024 * 1024
ray.shutdown()
ray.init(local_mode=True, memory=memory, 
        object_store_memory=object_store_memory,
        driver_object_store_memory=driver_object_store_memory,
        num_cpus=1)

# redis_password = sys.argv[1]
# num_cpus = int(sys.argv[2])

# ray.init(address=os.environ["ip_head"], redis_password=redis_password)

In [ ]:
config = {"num_workers": 1,
         "num_epochs":settings.num_epochs,
         "kfolds":flags.kfolds,
         "replacement_threshold": flags.replacement_threshold}

In [ ]:
# num_epochs = 10

In [ ]:
analysis = tune.run(
    trainable.train_AE, 
    name="exp_1",
    config=config,
    search_alg=algo,
    verbose=2, 
    resources_per_trial={
            "cpu": settings.num_cpus,
            "gpu": 0
    },
    num_samples=10,  #BUMP UP TO 1000
    scheduler=bohb,
    local_dir=settings.SAVE_FP+"TUNE_RESULT_DIR",
    stop=trainable.EarlyStopping("test_f1") # if search results aren't improving anymore
    )

#print("Best config is:", analysis.get_best_config(metric="test_f1"))

In [ ]:
print("Best config is:", analysis.get_best_config(metric="test_f1"))

In [ ]:
# for i in train_vars:
#     if isinstance(train_vars[i], dict):
#         print("Best "+i+":", analysis.get_best_config(metric="test_f1")[i])
for i in config_space:
    print("Best "+i+":", analysis.get_best_config(metric="test_f1")[i])

In [ ]:
# analysis = tune.run(
#     trainable.train_AE, 
#     name="exp_1",
#     config=train_vars, 
#     verbose=2, 
#     resources_per_trial={
#             "cpu": 2,
#             "gpu": 0
#     },
#     num_samples=2,
#     scheduler=ASHAScheduler(metric="test_f1", mode="max", grace_period=1, time_attr="n_batch"),
#     local_dir=settings.SAVE_FP+"TUNE_RESULT_DIR"
#     )

# print("Best config is:", analysis.get_best_config(metric="test_f1"))

In [ ]:
# analysis.dataframe()["logdir"][0]

In [ ]:
# train the model
# train_losses, test_losses, train_f1_scores, test_f1_scores = train_test.train_model(loaders, 
#         model, settings.num_epochs, flags.print_every,
#         settings.SAVE_FP, flags.replacement_threshold, cluster_names, flags.cirriculum, train_data[:,:len(cluster_names)],
#         search_space)
#train_losses, test_losses, train_f1_scores, test_f1_scores = train_test.train_model(train_vars, hyperparams)

### Evaluate model performance

In [ ]:
# # evaluate model performance
# perf_lc = data_viz.learning_curve(train_f1_scores, test_f1_scores, "performance", flags.cirriculum)

In [ ]:
# # evaluate model performance
# optim_lc = data_viz.learning_curve(train_losses, test_losses, "optimization", flags.cirriculum)

In [ ]:
# # first convert test_data from subset -> tensor, split corrupt vs target sets
# tensor_test_data = torch.tensor([i.numpy() for i in test_data]).float()
# corrupt_test_data = tensor_test_data[:,:num_features]
# target = tensor_test_data[:,num_features:].detach().numpy()

In [ ]:
# # Generate probabilities for ROC curve
# model.eval()
# with torch.no_grad():
#     y_probas = model(corrupt_test_data) # predicted probabilities generated by model

In [ ]:
# roc = data_viz.my_roc_curve(target, y_probas.numpy())

In [ ]:
# util.log_results(roc, optim_lc, perf_lc, flags, model)

In [ ]:
# # create embeddings for test set
# #uncorrupt_test_data = tensor_test_data[:,len(cluster_names):]
# #tensor_test_data = torch.tensor([i.numpy() for i in test_data]).float()
# embeddings = train_test.generate_embeddings(model, corrupt_test_data)

In [ ]:
#data_viz.plot_tSNE(embeddings, test_data, num_to_genome, genome_to_tax, test_tax_dict)

In [ ]:
# # tSNE for corrupted genomes passed through untrained model
# untrained_model = models.AutoEncoder(len(cluster_names))
# untr_embeddings = train_test.generate_embeddings(untrained_model, corrupt_test_data)

In [ ]:
#data_viz.plot_tSNE(untr_embeddings, test_data, num_to_genome, genome_to_tax, test_tax_dict)
# data_viz.plot_tSNE(untr_embeddings, test_data, num_to_genome, genome_to_tax, genome_idx_test)

In [ ]:
# # Evaluate model and compare against baselines
# # Get corrupted input set, target set, and predictions set (binarized to 1's and 0's)
# #corrupt_test_data = tensor_test_data[:,:len(cluster_names)]

# model.eval()
# with torch.no_grad():
#     pred = model.forward(corrupt_test_data).detach().numpy()
# b_pred = train_test.binarize(pred, flags.replacement_threshold)

In [ ]:
# # Generate confusion matrix
# cm = evaluate.dom_confusion_matrix(b_pred, target, num_to_genome, genome_to_tax, test_tax_dict, genome_idx_test)

In [ ]:
# util.log_results(roc, optim_lc, perf_lc, flags, model, cm)

In [ ]:
# # Baseline 1: untrained DAE
# # Generate predictions using an untrained DAE model
# model.eval()
# with torch.no_grad():
#     untr_pred = untrained_model.forward(corrupt_test_data).detach().numpy()
# untr_b_preds = train_test.binarize(untr_pred, flags.replacement_threshold)

In [ ]:
# # if os.path.isfile(settings.DATA_FP+"rand_b_pred.pt"):
# #     print("Loading random predictions from file")
# #     rand_b_pred = torch.load(settings.DATA_FP+"rand_b_pred.pt")
# # else: 
# #     # This is slow
# #     print("Generating random predictions, this will take a while (~30 min)")
# #     rand_b_pred = evaluate.generate_baseline(num_features, train_data, 
# #                                              corrupt_test_data, "base_random", cluster_names)
# #     torch.save(rand_b_pred, settings.DATA_FP+"rand_b_pred.pt")

# rand_b_pred = evaluate.generate_baseline(num_features, train_data, 
#                                          corrupt_test_data, "base_random", cluster_names)

In [ ]:
# torch.save(rand_b_pred, settings.DATA_FP+"rand_b_pred.pt")

In [ ]:
# # if os.path.isfile(settings.DATA_FP+"smart_b_pred.pt"):
# #     print("Loading smart random predictions from file")
# #     smart_b_pred = torch.load(settings.DATA_FP+"smart_b_pred.pt")
# # else:
# #     print("Generating smart random predictions, this will take a while (~30 min)")
# #     smart_b_pred = evaluate.generate_baseline(num_features, train_data, 
# #                                           corrupt_test_data, "smart_random", cluster_names)
# #     torch.save(smart_b_pred, settings.DATA_FP+"smart_b_pred.pt")

# smart_b_pred = evaluate.generate_baseline(num_features, train_data, 
#                                       corrupt_test_data, "smart_random", cluster_names)

In [ ]:
# torch.save(smart_b_pred, settings.DATA_FP+"smart_b_pred.pt")

In [ ]:
# import numpy as np
# np.sum(smart_b_pred == rand_b_pred), np.sum(smart_b_pred != rand_b_pred)

In [ ]:
# import pandas as pd
# hs = evaluate.hamming(target, b_pred)
# hs_stats = [round(sum(hs)/len(hs),2), round(min(hs),2), round(max(hs),2)]

# untr_hs = evaluate.hamming(target, untr_b_preds)
# untr_hs_stats = [round(sum(untr_hs)/len(untr_hs),2), round(min(untr_hs),2), round(max(untr_hs),2)]

# rand_hs = evaluate.hamming(target, rand_b_pred)
# rand_hs_stats = [round(sum(rand_hs)/len(rand_hs),2), round(min(rand_hs),2), round(max(rand_hs),2)]

# smart_hs = evaluate.hamming(target, smart_b_pred)
# smart_hs_stats = [round(sum(smart_hs)/len(smart_hs),2), round(min(smart_hs),2), round(max(smart_hs),2)]


# hamming_df = pd.DataFrame([hs_stats, untr_hs_stats, rand_hs_stats, smart_hs_stats], columns=['mean', 'min', 'max'], 
#                             index=["DAE trained", "DAE untrained", "Random chance", "Smart random chance"])
# hamming_df

In [ ]:
sys.stdout.flush()
sys.stderr.flush()